In [31]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip


--2021-12-17 07:31:12--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2021-12-17 07:31:12 (382 MB/s) - ‘datasource.zip’ saved [4249/4249]



In [7]:
!unzip datasource.zip -d dealership_data


Archive:  datasource.zip
replace dealership_data/used_car_prices1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


In [18]:
csv_files = glob.glob("dealership_data/*.csv")

json_files = glob.glob("dealership_data/*.json")

xml_files = glob.glob("dealership_data/*.xml")


['dealership_data/used_car_prices1.xml',
 'dealership_data/used_car_prices3.xml',
 'dealership_data/used_car_prices2.xml']

In [28]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

In [25]:
#extract fucntion
def extract_csv(csv_file):
  dataframe = pd.read_csv(csv_file)
  return dataframe

def extract_json(json_file):
  dataframe = pd.read_json(json_file, lines = True)
  return dataframe

def extract_xml(file_to_process):
    dataframe = pd.DataFrame(columns = ['car_model', 'year_of_manufacture', 'price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = (person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({'car_model': car_model, 'year_of_manufacture':year_of_manufacture, 'price':price, 'fuel':fuel}, ignore_index=True)
    return dataframe


In [26]:
def extract():
  extracted_data = pd.DataFrame(columns = ['car_model', 'year_of_manufacture', 'price', 'fuel'])
  for csv_file in csv_files:
    extracted_data = extracted_data.append(extract_csv(csv_file), ignore_index = True)

  for json_file in json_files:
    extracted_data = extracted_data.append(extract_json(json_file), ignore_index = True)

  for xml_file in xml_files:
    extracted_data = extracted_data.append(extract_xml(xml_file), ignore_index = True)

  
  
  
  return extracted_data

print(extract())


        car_model year_of_manufacture         price    fuel
0            ritz                2014   5000.000000  Petrol
1             sx4                2013   7089.552239  Diesel
2            ciaz                2017  10820.895522  Petrol
3         wagon r                2011   4253.731343  Petrol
4           swift                2014   6865.671642  Diesel
..            ...                 ...           ...     ...
85        etios g                2015   5895.522388  Petrol
86  corolla altis                2013   8208.955224  Petrol
87        corolla                2004   2238.805970  Petrol
88  corolla altis                2010   7835.820896  Petrol
89       fortuner                2012  21641.791045  Diesel

[90 rows x 4 columns]


In [27]:
def transform(data):
  # Round the price columns to 2 decimal places
  data["price"] = round(data.price , 2)
  return data

transform(extract())

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
...,...,...,...,...
85,etios g,2015,5895.52,Petrol
86,corolla altis,2013,8208.96,Petrol
87,corolla,2004,2238.81,Petrol
88,corolla altis,2010,7835.82,Petrol


In [30]:
def load(targetfile, source_data):
  source_data.to_csv("targetfile")

load(targetfile, transform(extract()))
  

In [53]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(logfile,"a") as f:
        f.write(timestamp + ',' + message + '\n')


In [52]:
log("ETL Job Started")

In [54]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
...,...,...,...,...
85,etios g,2015,5895.522388,Petrol
86,corolla altis,2013,8208.955224,Petrol
87,corolla,2004,2238.805970,Petrol
88,corolla altis,2010,7835.820896,Petrol


In [55]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
...,...,...,...,...
85,etios g,2015,5895.52,Petrol
86,corolla altis,2013,8208.96,Petrol
87,corolla,2004,2238.81,Petrol
88,corolla altis,2010,7835.82,Petrol


In [56]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [58]:
log("ETL Job Ended")